In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import random
import csv
import pandas as pd

In [ ]:
#Website URL INSERT LETTER WHERE THE # IS!! AFTER THE SLASH
url="https://www.azlyrics.com/#.html"

#collect artist names from page
page = requests.get(url)
soup = BeautifulSoup(page.content, 'lxml')
x = soup.find_all(class_="col-sm-6 text-center artist-col")

#initialize empty lists
links = []
artist_names = []

#fill link list and artist_names list
for col in x:
    g_tags = col.find_all("a")
    for item in g_tags:
        links.append(item.get('href'))
        artist_names.append(item.text)
        
#Create list of URLs
artist_urls = ["https://www.azlyrics.com/" + addition for addition in links]
urls = []
#albums = []
songTitles = []
artistNames = []
#lyrics = []
#year = []
counter = 0
z=list(zip(artist_names, artist_urls))
#print(z)
#Collect Data
allData = []
for item in z:
    artist_name = item[0]
    url = item[1]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'lxml')
    divs = soup.find_all("div")
    current_album=""
    for div in divs:
        if div.has_attr("class"):
            if div.attrs["class"][0]=="album":
                current_album = div.text
            elif div.attrs["class"][0] == "listalbum-item":
                endLink = div.a.get("href")
                songURL = "https://www.azlyrics.com/" + endLink[3:]
                curr_song = div.text
                page1 = requests.get(songURL)
                soupp = BeautifulSoup(page1.content, 'lxml')
                x = soupp.find('div', {'class': 'col-xs-12 col-lg-8 text-center'})
                y = x.find_all('div')
                curr_year = current_album[-5:-1]
                lyrics = y[5].text
                try:
                    year = int(curr_year)
                    yearUP = str(year)
                    Row = [artist_name, current_album, yearUP, curr_song, lyrics]
                    allData.append(Row)
                    print(Row[:4])
                    counter += 1
                    print(str(350-counter) + ' Songs Left')
                except:
                    print('Invalid Year, Ignoring Song')
                    
                t = random.uniform(4,10)
                print('Pausing ' + str(t) + ' Seconds')
                print(counter)
                time.sleep(t)
                
        if counter == 350:
            break
    if counter == 350:
        break
                
            
with open('songLyricsALL.csv', 'w', encoding="utf-8", newline='') as myfile:
    wr = csv.writer(myfile)
    wr.writerow(("Artist", "Album", "Year", "Song Title", "Lyrics"))
    wr.writerows(allData)
myfile.close()
